In [1]:
import numpy as np
import pandas as pd
import streamlit as st
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
tqdm.pandas() 

# 파일불러오기
file_path_10 = './data/대여소별 대여반납 승객수 정보(202310월).csv'
file_path_11 = './data/대여소별 대여반납 승객수 정보(202311월).csv'
file_path_12 = './data/대여소별 대여반납 승객수 정보(202312월).csv'
file_path_4 = './data/tpss_bcycl_od_statnhm_20240104.csv'
file_path_5 = './data/tpss_bcycl_od_statnhm_20240105.csv'
file_path_6 = './data/tpss_bcycl_od_statnhm_20240106.csv'
file_path_master = './data/서울시 따릉이대여소 마스터 정보.csv'
df_rent10 = pd.read_csv(file_path_10,encoding='cp949')
df_rent11 = pd.read_csv(file_path_11,encoding='cp949')
df_rent12 = pd.read_csv(file_path_12,encoding='cp949')
df_master = pd.read_csv(file_path_master,encoding='cp949')
df_rent4 = pd.read_csv(file_path_4,encoding='cp949')
df_rent5 = pd.read_csv(file_path_5,encoding='cp949')
df_rent6 = pd.read_csv(file_path_6,encoding='cp949')

In [11]:
df_sample = df_rent12.iloc[:1000,:]
# df_sample
file_path = './data/rent_sample.csv'
df_sample.to_csv(file_path,encoding='utf-8',index=False)

In [12]:
df_sample1 = pd.read_csv(file_path)
df_sample1

,기준_날짜,집계_기준,기준_시간대,시작_대여소_ID,시작_대여소명,종료_대여소_ID,종료_대여소명,전체_건수,전체_이용_분,전체_이용_거리
0,20231219,NaN,0,ST-1045,NaN,ST-1083,NaN,1,59,710
1,20231219,NaN,0,ST-1063,NaN,ST-1063,NaN,1,0,0
2,20231219,NaN,0,ST-1092,NaN,ST-502,NaN,1,6,1135
3,20231219,NaN,0,ST-1093,NaN,ST-566,NaN,1,3,680
4,20231219,NaN,0,ST-1124,NaN,ST-2967,NaN,1,8,1710
...,...,...,...,...,...,...,...,...,...,...
995,20231219,NaN,25,ST-2876,NaN,ST-550,NaN,1,8,1500
996,20231219,NaN,25,ST-2905,NaN,ST-2167,NaN,1,12,2200
997,20231219,NaN,25,ST-2938,NaN,ST-895,NaN,1,8,1436
998,20231219,NaN,25,ST-295,NaN,ST-293,NaN,1,3,492


In [2]:
# 분석할 3개월 데이터 합치기
df_rent = pd.concat([df_rent10,df_rent11,df_rent12])
# 데이터가 너무 많아서 10월분으로만 진행
# df_rent = df_rent12

# 3일 렌트데이터 합치기 : 대여소명을 가져오기 위한 용도
df_rent456 = pd.concat([df_rent4,df_rent5,df_rent6])

In [3]:
# 전체이용거리가 0인 행 제거
cond = (df_rent['전체_이용_거리']==0)
df_rent.drop(df_rent.loc[cond,:].index, inplace=True)
df_rent.loc[cond,:]

,기준_날짜,집계_기준,기준_시간대,시작_대여소_ID,시작_대여소명,종료_대여소_ID,종료_대여소명,전체_건수,전체_이용_분,전체_이용_거리


In [4]:
df_rent.isnull().sum()

기준_날짜              0
집계_기준        3513604
기준_시간대             0
시작_대여소_ID          0
시작_대여소명      3513604
종료_대여소_ID          0
종료_대여소명      3513604
전체_건수              0
전체_이용_분            0
전체_이용_거리           0
dtype: int64

In [5]:
# 대여소ID별로 sum
df_rent_groupby = df_rent.groupby(by=['기준_날짜','시작_대여소_ID','종료_대여소_ID'])\
    .agg({'전체_건수':sum,'전체_이용_분':sum,'전체_이용_거리':sum})\
    .sort_values(by=['기준_날짜','시작_대여소_ID','종료_대여소_ID'])
df_rent_groupby.reset_index(inplace=True)
df_rent_groupby['기준_날짜'] = df_rent_groupby['기준_날짜'].astype('str')
df_rent_groupby.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1131098 entries, 0 to 1131097
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   기준_날짜      1131098 non-null  object
 1   시작_대여소_ID  1131098 non-null  object
 2   종료_대여소_ID  1131098 non-null  object
 3   전체_건수      1131098 non-null  int64 
 4   전체_이용_분    1131098 non-null  int64 
 5   전체_이용_거리   1131098 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 51.8+ MB


In [6]:
# df_master '위도','경도' 컬럼명 변경
df_master.rename(columns={'위도':'lat','경도':'lon'},inplace=True)
# df_master '위도','경도' 정보가 없는 행 삭제
cond = ((df_master['lat']==0) | (df_master['lon']==0))  # 77개행
df_master.drop(df_master.loc[cond,:].index,inplace=True,axis=0)
df_master

,대여소_ID,주소1,주소2,lat,lon
2,ST-997,서울특별시 양천구 목동중앙로 49,목동3단지 시내버스정류장,37.534390,126.869598
3,ST-996,서울특별시 양천구 남부순환로88길5-16,양강중학교앞 교차로,37.524334,126.850548
4,ST-995,서울특별시 양천구 중앙로 153 공중화장실,NaN,37.510597,126.857323
5,ST-994,서울특별시 양천구 목동서로161,SBS방송국,37.529163,126.872749
6,ST-993,서울특별시 양천구 신월로 342-1 구두수선대19,NaN,37.521511,126.857384
...,...,...,...,...,...
3264,ST-1003,서울특별시 양천구 화곡로 59,신월동 이마트,37.539551,126.828300
3265,ST-1002,서울특별시 양천구 목동동로 316-6,서울시 도로환경관리센터,37.529900,126.876541
3267,ST-1000,서울특별시 양천구 신정동 236,서부식자재마트 건너편,37.510380,126.866798
3268,ST-100,서울특별시 광진구 아차산로 262,더샵스타시티 C동 앞,37.536667,127.073593


In [7]:
# 대여소ID, 대여소명 마스터 만들기
df_rent_id_master = df_rent456.drop_duplicates(subset='시작_대여소_ID').loc[:,['시작_대여소_ID','시작_대여소명']]
# df_rent_id_master = df_rent_id_master.loc[:,['시작_대여소_ID','시작_대여소명']]
df_rent_id_master.rename(columns={'시작_대여소_ID':'대여소_ID','시작_대여소명':'대여소명'},inplace=True)
df_rent_id_master.reset_index(inplace=True,drop=True)
df_rent_id_master

,대여소_ID,대여소명
0,ST-1014,신정3동_101_1
1,ST-1017,목5동_059_1
2,ST-1055,강일동_001_1
3,ST-1082,장지동_019_1
4,ST-1092,풍납2동_026_1
...,...,...
2717,ST-778,세곡동_007_2
2718,ST-1659,성수1가2동_001_2
2719,ST-2939,위례동_001_3
2720,ST-3216,내곡동_007_3


In [8]:
# 사용자 함수 정의
# 마스터정보의 시작_대여소명를  대여소이용정보에 추가
def id_to_name(id):
    cond_id = (df_rent_id_master['대여소_ID']==id)
    
    if len(df_rent_id_master.loc[cond_id,'대여소명'].values)>0:
        id_name = df_rent_id_master.loc[cond_id,'대여소명'].values[0]
    else:
        id_name = 'no_data'
        
    return id_name

In [9]:
# print(id_to_name('ST-100'))
df_rent_groupby['시작_대여소명'] = df_rent_groupby['시작_대여소_ID'].progress_apply(lambda x:id_to_name(x))
df_rent_groupby['종료_대여소명'] = df_rent_groupby['종료_대여소_ID'].progress_apply(lambda x:id_to_name(x))
file_path_groupby = './data/groupby_23년12월렌탈.csv'
df_rent_groupby.to_csv(file_path_groupby)

  0%|          | 0/1131098 [00:00<?, ?it/s]

100%|██████████| 1131098/1131098 [13:19<00:00, 1414.92it/s]


In [11]:
# 사용자 함수 정의
# 마스터정보의 경도, 위도 정보를  대여소이용정보에 추가
def find_lat_lon(id):
    # 종료_대여소_ID의 경우 'X'인 경우 처리
    if id=='X':
        ret_start = pd.Series(['0','0'])
    else:
        cond_start = (df_master['대여소_ID']==id)
        ret_start = df_master.loc[cond_start,['lat','lon']].values[0]

    return pd.Series([ret_start[0], ret_start[1]])

In [22]:
# df_rent1에 경도,위도 정보 추가
df_rent_groupby[['시작_대여소_lat','시작_대여소_lon']] = df_rent_groupby['시작_대여소_ID'].progress_apply(lambda x:find_lat_lon(x))
df_rent_groupby[['종료_대여소_lat','종료_대여소_lon']] = df_rent_groupby['종료_대여소_ID'].progress_apply(lambda x:find_lat_lon(x))
df_rent_groupby_lat = df_rent_groupby

100%|██████████| 1131098/1131098 [24:38<00:00, 764.78it/s]


In [24]:
file_path_groupby_lat = './data/groupby_lat_23년12월렌탈.csv'
df_rent_groupby_lat.to_csv(file_path_groupby_lat)

In [29]:
df_featured_rent.head(n=2)

,기준_날짜,시작_대여소_ID,종료_대여소_ID,전체_건수,전체_이용_분,전체_이용_거리,시작_대여소명,종료_대여소명,시작_대여소_lat,시작_대여소_lon,종료_대여소_lat,종료_대여소_lon,시작_대여소_동명,종료_대여소_동명
0,20231201,ST-10,ST-10,2,218,2780,서교동_037_1,서교동_037_1,37.552746,126.918617,37.552746,126.918617,서교동,서교동
1,20231201,ST-10,ST-11,2,22,1954,서교동_037_1,서강동_046_1,37.552746,126.918617,37.547691,126.919983,서교동,서강동


In [28]:
df_featured_rent = df_rent_groupby_lat
df_featured_rent['시작_대여소_동명'] = df_rent_groupby_lat['시작_대여소명'].apply(lambda x:x[:x.find('동')+1])
df_featured_rent['종료_대여소_동명'] = df_rent_groupby_lat['종료_대여소명'].apply(lambda x:x[:x.find('동')+1])

In [31]:
file_path_groupby_lat_dong = './data/groupby_lat_dong_23년12월렌탈.csv'
df_featured_rent.to_csv(file_path_groupby_lat_dong)